In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.constants import Rd
from metpy.units import units, concatenate
%matplotlib inline

In [2]:
df = pd.read_csv('Soundings_1992.txt', index_col=0)

/Users/johnleeman/miniconda3/envs/unidata-workshop/lib/python3.6/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df.head()

,Date,Time,Station,Pressure,Temperature,Dewpoint,u wind,v wind,Speed,Direction
0,1992-01-01,0,UIL,1012.0,NaN,NaN,-4.000000,6.928203,8.0,150.0
1,1992-01-01,0,UIL,1000.0,10.2,3.2,-5.916656,12.688309,14.0,155.0
2,1992-01-01,0,UIL,982.9,9.3,2.9,-7.182423,19.733545,21.0,160.0
3,1992-01-01,0,UIL,947.4,7.3,2.3,-5.556742,31.513848,32.0,170.0
4,1992-01-01,0,UIL,913.2,5.4,1.8,-3.573385,40.843983,41.0,175.0


In [4]:
def cape_cin(pressure, temperature, dewpt, parcel_profile):
    r"""Calculate convective available potential energy (CAPE).
        Description
        Parameters
        ----------
        pressure : `pint.Quantity`
            The atmospheric pressure level(s) of interest. The first entry should be the starting
            point pressure.
        temperature : `pint.Quantity`
            The starting temperature
        dewpt : `pint.Quantity`
            The starting dew point
        parcel_profile : `pint.Quantity`
            The temperature profile of the parcel
        Returns
        -------
        `pint.Quantity`
            Convective available potential energy (CAPE).
        `pint.Quantity`
            Convective inhibition (CIN).
        See Also
        --------
        lfc, el
        """
    # Calculate limits of integration
    lfc_pressure = mpcalc.lfc(pressure, temperature, dewpt)[0]
    el_pressure = mpcalc.el(pressure, temperature, dewpt)[0]

    # If there is no LFC, no need to proceed.
    if lfc_pressure is None:
        return 0 * units('J/kg')
    else:
        lfc_pressure = lfc_pressure.magnitude

    # No EL and we use the top reading of the sounding.
    if el_pressure is None:
        el_pressure = pressure[-1].magnitude
    else:
        el_pressure = el_pressure.magnitude

    # Difference between the parcel path and measured temperature profiles
    y = (parcel_profile - temperature).to(units.degK)
    
    # Estimate zero crossings
    x,y = _find_append_zero_crossings(np.copy(pressure),y)
    
    # CAPE (temperature parcel < temperature environment)
    # Only use data between the LFC and EL for calculation
    p_mask = (x <= lfc_pressure) & (x >= el_pressure)
    x_clipped = x[p_mask]
    y_clipped = y[p_mask]
    
    y_clipped[y_clipped <= 0 * units.degK] = 0 * units.degK
    cape = (Rd * (np.trapz(y_clipped, np.log(x_clipped)) * units.degK)).to(units('J/kg'))
    
    # CIN (temperature parcel < temperature environment)
    # Only use data between the surface and LFC for calculation
    p_mask = (x >= lfc_pressure)
    x_clipped = x[p_mask]
    y_clipped = y[p_mask]
    
    y_clipped[y_clipped >= 0 * units.degK] = 0 * units.degK
    cin = (Rd * (np.trapz(y_clipped, np.log(x_clipped)) * units.degK)).to(units('J/kg'))
    
    return cape, cin

In [5]:
def parcel_profile_tv(p, T, Td):
    prof = mpcalc.parcel_profile(p, T[0], Td[0]).to('degC')
    #Find dewpoint for the virtual temperature calculation for the parcel
    #Below LCL: dewpoint = environmental dewpoint
    #Above LCL: dewpoint = parcel temp, since it's saturated.
    lcl_pressure,_ = mpcalc.lcl(p[0], T[0], Td[0])
    parcel_dewpoint = Td
    idx = np.where(p <= lcl_pressure)
    parcel_dewpoint[idx] = T[idx]
    
    Rv = mpcalc.mixing_ratio(mpcalc.saturation_vapor_pressure(parcel_dewpoint), p)
    return mpcalc.virtual_temperature(T, Rv)

In [ ]:
def _find_append_zero_crossings(x, y):
    r"""
    Find and interpolate zero crossings
    Estimate the zero crossings of an x,y series and add estimated crossings to series,
    returning a sorted array with no duplicate values.
    Parameters
    ----------
    x : `pint.Quantity`
        x values of data
    y : `pint.Quantity`
        y values of data
    Returns
    -------
    x : `pint.Quantity`
        x values of data
    y : `pint.Quantity`
        y values of data
    """
    # Find and append crossings to the data
    crossings = mpcalc.find_intersections(x[1:], y[1:], np.zeros_like(y[1:]))
    x = concatenate((x, crossings[0]))
    y = concatenate((y, crossings[1]))

    # Resort so that data are in order
    sort_idx = np.argsort(x)
    x = x[sort_idx]
    y = y[sort_idx]

    # Remove duplicate data points if there are any
    keep_idx = np.ediff1d(x, to_end=[1]) > 0
    x = x[keep_idx]
    y = y[keep_idx]
    return x, y

In [ ]:
calculations = []

for name, group in df.groupby(['Date', 'Time', 'Station']):
    print(name)
    
    # Pull out data and put units back on it
    p = np.array(group['Pressure']) * units.hPa
    T = np.array(group['Temperature']) * units.degC 
    Td = np.array(group['Dewpoint']) * units.degC
    
    # CAPE/CIN with no virtual temperature correction
    try:
        parcel_path = mpcalc.parcel_profile(p, T[0], Td[0])
        cape, cin= cape_cin(p, T, Td, parcel_path)
        cape = cape.m
        cin = cin.m
    except:
        cape = 0
        cin = 0
        print("ERROR IN CAPE/CIN", name)
        
    # CAPE/CIN with virtual temperature correction
    try:
        parcel_path = parcel_profile_tv(p, T, Td)
        cape_tv, cin_tv= cape_cin(p, T, Td, parcel_path)
        cape_tv = cape_tv.m
        cin_tv = cin_tv.m
    except:
        cape_tv = 0
        cin_tv = 0
        print("ERROR IN CAPE/CIN TV", name)
    
    calculations.append({'Date': name[0], 
                         'Time': name[1], 
                         'Station': name[2], 
                         'CAPE': cape, 
                         'CIN': cin, 
                         'CAPE_Tv': cape_tv,
                         'CIN_Tv': cin_tv})

('1992-01-01', 0, 'ABQ')
('1992-01-01', 0, 'ALB')


/Users/johnleeman/miniconda3/envs/unidata-workshop/lib/python3.6/site-packages/pint/quantity.py:1065: RuntimeWarning: invalid value encountered in less_equal
  return op(self._convert_magnitude_not_inplace(UnitsContainer()), other)
/Users/johnleeman/miniconda3/envs/unidata-workshop/lib/python3.6/site-packages/pint/quantity.py:1065: RuntimeWarning: invalid value encountered in greater_equal
  return op(self._convert_magnitude_not_inplace(UnitsContainer()), other)


('1992-01-01', 0, 'AMA')
('1992-01-01', 0, 'BIS')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'BIS')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'BIS')
('1992-01-01', 0, 'BNA')


/Users/johnleeman/miniconda3/envs/unidata-workshop/lib/python3.6/site-packages/metpy/calc/thermo.py:190: RuntimeWarning: invalid value encountered in absolute
  if np.abs(new_p - p).max() < eps:
/Users/johnleeman/miniconda3/envs/unidata-workshop/lib/python3.6/site-packages/pint/quantity.py:1403: RuntimeWarning: invalid value encountered in absolute
  out = uf(*mobjs)


('1992-01-01', 0, 'BOI')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'BOI')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'BOI')
('1992-01-01', 0, 'BUF')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'BUF')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'BUF')
('1992-01-01', 0, 'CAR')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'CAR')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'CAR')
('1992-01-01', 0, 'CHH')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'CHH')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'CHH')
('1992-01-01', 0, 'CHS')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'CHS')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'CHS')
('1992-01-01', 0, 'CRP')
('1992-01-01', 0, 'DDC')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'DDC')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'DDC')
('1992-01-01', 0, 'DNR')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'DNR')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'DNR')
('1992-01-01', 0, 'DRT')
('1992-01-01', 0, 'GGW')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'GGW')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'GGW')
('1992-01-01', 0, 'GJT')
('1992-01-01'

...

('1992-06-03', 0, 'WAL')
('1992-06-03', 0, 'WMW')
('1992-06-03', 0, 'YMO')
('1992-06-03', 0, 'YQI')
('1992-06-03', 12, 'ABQ')
('1992-06-03', 12, 'ADN')
('1992-06-03', 12, 'AMA')
('1992-06-03', 12, 'BIS')
('1992-06-03', 12, 'BNA')
('1992-06-03', 12, 'BOI')
('1992-06-03', 12, 'BRO')
('1992-06-03', 12, 'BUF')
('1992-06-03', 12, 'CAR')
('1992-06-03', 12, 'CHH')
('1992-06-03', 12, 'CHS')
('1992-06-03', 12, 'CRP')
('1992-06-03', 12, 'DDC')
('1992-06-03', 12, 'DNR')
('1992-06-03', 12, 'DRT')
('1992-06-03', 12, 'GGW')
('1992-06-03', 12, 'GJT')
('1992-06-03', 12, 'GRB')
('1992-06-03', 12, 'GSO')
('1992-06-03', 12, 'IAD')
('1992-06-03', 12, 'INL')
('1992-06-03', 12, 'JAN')
('1992-06-03', 12, 'LBF')
('1992-06-03', 12, 'LCH')
('1992-06-03', 12, 'LIX')
('1992-06-03', 12, 'LZK')
('1992-06-03', 12, 'MAF')
('1992-06-03', 12, 'MFR')
('1992-06-03', 12, 'NKX')
('1992-06-03', 12, 'OAK')
('1992-06-03', 12, 'OUN')
('1992-06-03', 12, 'PIT')
('1992-06-03', 12, 'RAP')
('1992-06-03', 12, 'SLC')
('1992-06-03', 1

('1992-06-07', 0, 'AMA')
('1992-06-07', 0, 'BIS')
('1992-06-07', 0, 'BNA')
ERROR IN CAPE/CIN ('1992-06-07', 0, 'BNA')
ERROR IN CAPE/CIN TV ('1992-06-07', 0, 'BNA')
('1992-06-07', 0, 'BOI')
('1992-06-07', 0, 'BRO')
ERROR IN CAPE/CIN ('1992-06-07', 0, 'BRO')
ERROR IN CAPE/CIN TV ('1992-06-07', 0, 'BRO')
('1992-06-07', 0, 'BUF')
('1992-06-07', 0, 'CAR')
('1992-06-07', 0, 'CHH')
('1992-06-07', 0, 'CHS')
('1992-06-07', 0, 'CRP')
('1992-06-07', 0, 'DDC')
('1992-06-07', 0, 'DNR')
('1992-06-07', 0, 'DRT')
('1992-06-07', 0, 'GGW')
('1992-06-07', 0, 'GJT')
('1992-06-07', 0, 'GRB')
('1992-06-07', 0, 'GSO')
('1992-06-07', 0, 'IAD')
('1992-06-07', 0, 'INL')
('1992-06-07', 0, 'JAN')
('1992-06-07', 0, 'LBF')
('1992-06-07', 0, 'LCH')
('1992-06-07', 0, 'LIX')
('1992-06-07', 0, 'LZK')
('1992-06-07', 0, 'MFR')
('1992-06-07', 0, 'NKX')
('1992-06-07', 0, 'OAK')
('1992-06-07', 0, 'OUN')
('1992-06-07', 0, 'PIT')
('1992-06-07', 0, 'RAP')
('1992-06-07', 0, 'SLC')
('1992-06-07', 0, 'TBW')
('1992-06-07', 0, 'TLH

('1992-06-10', 12, 'CHS')
('1992-06-10', 12, 'CRP')
('1992-06-10', 12, 'DDC')
('1992-06-10', 12, 'DNR')
('1992-06-10', 12, 'DRT')
ERROR IN CAPE/CIN ('1992-06-10', 12, 'DRT')
ERROR IN CAPE/CIN TV ('1992-06-10', 12, 'DRT')
('1992-06-10', 12, 'GGW')
('1992-06-10', 12, 'GJT')
('1992-06-10', 12, 'GRB')
('1992-06-10', 12, 'GSO')
('1992-06-10', 12, 'IAD')
('1992-06-10', 12, 'INL')
('1992-06-10', 12, 'JAN')
('1992-06-10', 12, 'LBF')
('1992-06-10', 12, 'LCH')
('1992-06-10', 12, 'LIX')
('1992-06-10', 12, 'LZK')
('1992-06-10', 12, 'MFR')
('1992-06-10', 12, 'NKX')
('1992-06-10', 12, 'OAK')
('1992-06-10', 12, 'OUN')
('1992-06-10', 12, 'PIT')
('1992-06-10', 12, 'RAP')
('1992-06-10', 12, 'SLC')
('1992-06-10', 12, 'TBW')
('1992-06-10', 12, 'TLH')
('1992-06-10', 12, 'TOP')
('1992-06-10', 12, 'TUS')
('1992-06-10', 12, 'UIL')
('1992-06-10', 12, 'WAL')
('1992-06-10', 12, 'WMW')
('1992-06-10', 12, 'WPL')
('1992-06-10', 12, 'YMO')
('1992-06-10', 12, 'YQI')
('1992-06-11', 0, 'ABQ')
('1992-06-11', 0, 'ALB')
(

('1992-06-14', 0, 'CHH')
('1992-06-14', 0, 'CHS')
('1992-06-14', 0, 'DDC')
('1992-06-14', 0, 'DNR')
('1992-06-14', 0, 'DRT')
('1992-06-14', 0, 'GGW')
('1992-06-14', 0, 'GJT')
('1992-06-14', 0, 'GRB')
('1992-06-14', 0, 'GSO')
('1992-06-14', 0, 'IAD')
('1992-06-14', 0, 'INL')
('1992-06-14', 0, 'JAN')
('1992-06-14', 0, 'LBF')
('1992-06-14', 0, 'LCH')
('1992-06-14', 0, 'LIX')
('1992-06-14', 0, 'LZK')
('1992-06-14', 0, 'MAF')
('1992-06-14', 0, 'MFR')
('1992-06-14', 0, 'NKX')
('1992-06-14', 0, 'OAK')
('1992-06-14', 0, 'OUN')
('1992-06-14', 0, 'PIT')
('1992-06-14', 0, 'RAP')
('1992-06-14', 0, 'SLC')
('1992-06-14', 0, 'TBW')
('1992-06-14', 0, 'TLH')
('1992-06-14', 0, 'TOP')
('1992-06-14', 0, 'TUS')
('1992-06-14', 0, 'UIL')
('1992-06-14', 0, 'WAL')
('1992-06-14', 0, 'WMW')
('1992-06-14', 0, 'WPL')
('1992-06-14', 0, 'YMO')
('1992-06-14', 0, 'YQI')
('1992-06-14', 12, 'ABQ')
('1992-06-14', 12, 'ADN')
('1992-06-14', 12, 'ALB')
('1992-06-14', 12, 'AMA')
('1992-06-14', 12, 'BIS')
('1992-06-14', 12, '

('1992-06-17', 12, 'TBW')
('1992-06-17', 12, 'TLH')
('1992-06-17', 12, 'TOP')
('1992-06-17', 12, 'TUS')
('1992-06-17', 12, 'UIL')
('1992-06-17', 12, 'WAL')
('1992-06-17', 12, 'WMW')
('1992-06-17', 12, 'WPL')
('1992-06-17', 12, 'YMO')
('1992-06-17', 12, 'YQI')
('1992-06-18', 0, 'ABQ')
('1992-06-18', 0, 'ALB')
('1992-06-18', 0, 'AMA')
('1992-06-18', 0, 'BIS')
('1992-06-18', 0, 'BNA')
('1992-06-18', 0, 'BOI')
('1992-06-18', 0, 'BRO')
('1992-06-18', 0, 'BUF')
('1992-06-18', 0, 'CHH')
('1992-06-18', 0, 'CHS')
('1992-06-18', 0, 'CRP')
('1992-06-18', 0, 'DDC')
('1992-06-18', 0, 'DNR')
('1992-06-18', 0, 'DRT')
('1992-06-18', 0, 'GGW')
('1992-06-18', 0, 'GJT')
('1992-06-18', 0, 'GRB')
('1992-06-18', 0, 'GSO')
('1992-06-18', 0, 'IAD')
('1992-06-18', 0, 'INL')
('1992-06-18', 0, 'JAN')
('1992-06-18', 0, 'LBF')
('1992-06-18', 0, 'LCH')
('1992-06-18', 0, 'LIX')
('1992-06-18', 0, 'LZK')
('1992-06-18', 0, 'MAF')
('1992-06-18', 0, 'MFR')
('1992-06-18', 0, 'NKX')
('1992-06-18', 0, 'OAK')
('1992-06-18', 

In [ ]:
# Empty dataframe to hold the results of our calculations
#calc_df = pd.DataFrame(columns=['Date', 'Time', 'Station', 'CAPE', 'CIN', 'CAPE_Tv', 'CIN_Tv'])

# Put our list of results into the dataframe
calc_df = pd.DataFrame(calculations)

In [ ]:
calc_df.head()

In [ ]:
calc_df.to_csv('1992_Calculations.txt')